In [ ]:
import pandas as pd
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

# === 1. Загружаем каунты, выделяем все гены по Ensembl ID ===
counts = pd.read_csv('<file_name>.csv', index_col=0) #загружаем матрицу каунтов с генами в формате Ensembl ID из TCGA
universe_ids = set(counts.index)
print(f"Всего генов в матрице каунтов: {len(universe_ids)}")

# === 2. Загружаем DEG-таблицу ===
deg_df = pd.read_excel('<file_name>.xlsx') #загружаем список ДЭГов (формат файла доступен в suplementary files ("DEGs..."))
deg_ids = set(deg_df['id'])
deg_symbols = set(deg_df['symbol'])
print(f"Количество DEGs: {len(deg_ids)}")

# === 3. Загружаем сигнатуры ===
signatures_df = pd.read_excel('<file_name>.xlsx') #загружаем в нужном формате таблицы с сигнатурами из литературы и предпроцессируем
signature_dict = signatures_df.groupby('signature')['genes'].apply(set).to_dict()

# === 4. Фишер по каждой сигнатуре ===
results = []

for signature_name, gene_set in signature_dict.items():
    gene_set = set(gene_set)

    # a: DEG ∩ подпиcь (по символам)
    a = len(deg_symbols & gene_set)

    # b: DEG, не входящие в категорию
    b = len(deg_ids) - a

    # c: не-DEG, но входящие в категорию
    c = len(gene_set) - a

    # d: остальные (все гены из counts, минус те, кто попал в категорию)
    d = len(universe_ids) - len(deg_ids) - c

    contingency = [[a, b], [c, d]]

    print(f"\n=== {signature_name} ===")
    print(f"Гены в категории: {len(gene_set)}")
    print(f"DEG ∩ Категория ({a}): {sorted(deg_symbols & gene_set)}")
    print(f"Таблица сопряженности: {contingency}")

    if a + b + c + d == 0:
        continue

    odds_ratio, pval = fisher_exact(contingency, alternative='greater')

    results.append({
        'signature': signature_name,
        'a': a, 'b': b, 'c': c, 'd': d,
        'contingency_table': contingency,
        'odds_ratio': odds_ratio,
        'pval': pval
    })

# === 5. Коррекция на множественное сравнение
results_df = pd.DataFrame(results)
results_df['padj'] = multipletests(results_df['pval'], method='fdr_bh')[1]
results_df = results_df.sort_values('padj')

# === 6. Вывод
pd.set_option('display.max_colwidth', None)
print("\n===Результаты обогащения===")
print(results_df[['signature', 'contingency_table', 'odds_ratio', 'pval', 'padj']])
